# Touch Screen Example Model

## Run one mapdl instance to build touch screen model and write node coordinates to file 

Make imports

In [ ]:
from ansys.mapdl.core import launch_mapdl
import os

# For quite exiting on error
from commons import StopExecution

# To mute annoying warnings in notebook
import warnings

# Implement warning muting
warnings.filterwarnings("ignore")

Run MAPDL instance

In [ ]:
try:
    mapdl = launch_mapdl(
        remove_temp_dir_on_exit=True, additional_switches="-smp", nproc=2
    )
except Exception:
    print("No MAPDL found. Terminating...")

    raise StopExecution

Define position of sensors

In [ ]:
Length_Inches = 4.5
Height_Inches = 2

sensors = {
    "x1": 1,
    "y1": 0.5,
    "x2": 2.25,
    "y2": 0.5,
    "x3": 3.5,
    "y3": 0.5,
    "x4": 1,
    "y4": 1.5,
    "x5": 2.25,
    "y5": 1.5,
    "x6": 3.5,
    "y6": 1.5,
    "x7": 0.5,
    "y7": 1,
    "x8": 4,
    "y8": 1,
}

sensors

Create geometry model

In [ ]:
# Clear model
mapdl.clear()

# Enter pre-processor
mapdl.prep7()

# Create a rectangular area or block volume by corner points
mapdl.blc4(0, 0, Length_Inches, Height_Inches, 0)

# Delete un-meshed areas
mapdl.adele("all")

# Generate a fillet line between two intersecting lines
mapdl.lfillt(1, 2, 0.2)
mapdl.lfillt(2, 3, 0.2)
mapdl.lfillt(3, 4, 0.2)
mapdl.lfillt(4, 1, 0.2)

# Compress the numbering of defined items
mapdl.numcmp("area")

# Generate an area bounded by previously defined lines
mapdl.al("all")

# Plot lines
mapdl.lplot(vtk=False)

In [ ]:
# Define hard points by sensor coordinates
for points in range(1, int((len(sensors) / 2) + 1)):
    mapdl.hptcreate(
        type_="area",
        entity=1,
        nhp="",
        label="coord",
        val1=sensors["x" + str(points)],
        val2=sensors["y" + str(points)],
    )

# Plot created area
mapdl.aplot()

Create FEA model

In [ ]:
# Set element type
mapdl.et(1, 181)

# Set material properties
mapdl.mp("ex", 1, 1e7)

# Set section type (shell representation)
mapdl.sectype(1, "shell")

# Set shell thickness
mapdl.secdata(0.1, 1)

# Specify the element shape as 2D
mapdl.mshape(1, "2d")

# Specify the element size to be meshed onto areas
mapdl.aesize("all", 0.05)

# Generate nodes and area elements within areas
mapdl.amesh("all")

# Select all entities with a single command
mapdl.allsel()

# Select those nodes associated with the selected lines
mapdl.nsll("u", 1)

# Select those nodes associated with the selected key points
mapdl.nslk("u")

# Get node count
node_count = int(mapdl.get_value("node", 0, "count"))

# Print node count
print(f"Node count: {node_count}")

In [ ]:
# Plot mesh
mapdl.eplot(vtk=True)

In [ ]:
# Get node coordinates
nodes = mapdl.mesh.nodes

nodes

In [ ]:
# Set path to file
result_path = r"sources\point_set.csv"

# Set path to save data
save_path = os.path.join(os.path.dirname(os.getcwd()), result_path)

# Write X,Y node coordinates to file
with open(save_path, "w") as file:
    for node in nodes:
        file.write(str(node[0]) + " , " + str(node[1]) + "\n")

In [ ]:
mapdl.exit()